In [ ]:
detr_path = '/home/sworkspace/results/ab_car_validation_germany/riders/Co-DETR/2w_misses_codetr/co_detr.tsv'

In [ ]:
import glob
import pandas as pd
import pickle
import os

pickles_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR/raw_detections/pickles'
output_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR'

def parse_pickles_to_dataframe():
    data = []
    for path in glob.iglob(f"{pickles_path}/*.pickle"):
        with open(path, 'rb') as file:
            detections = pickle.load(file)
            for detection in detections:
                bbox_str = str(detection.bbox)
                coords_str = bbox_str[bbox_str.index('<')+1:bbox_str.index('>')].split(',')[0:4]
                x1, y1, x2, y2 = map(float, [coord.strip("()") for coord in coords_str])
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2
                width = x2 - x1
                height = y2 - y1
                label = detection.category.id
                if label in [0, 1, 2, 3, 5, 7]:
                    if label in [1, 3]:
                        label = 1
                    elif label in [2, 5, 7]:
                        label = 2
                    score = detection.score.value*100
                    filename = os.path.basename(path)
                    data.append((filename.replace('.pickle', '.png'), x_center, y_center, width, height, label, score))
                    
    df = pd.DataFrame(data, columns=['name', 'x_center', 'y_center', 'width', 'height', 'label', 'score'])
    df[['is_occluded', 'is_truncated', 'd3_separation', 'is_rider_on_2_wheels']] = 0
    df[['l_label', 'r_label']] = 'None'
    df['threshold'] = 'None'
    df.to_csv(f"{output_path}/co_detr.tsv", sep='\t',index=False)
    print(f"{output_path}/co_detr.tsv")

In [ ]:
parse_pickles_to_dataframe()

In [ ]:
import pandas as pd

In [ ]:
detr = pd.read_csv(f"{output_path}/co_detr.tsv", sep='\t')
detr['label'].value_counts()

In [ ]:
det = pd.read_csv('/home/ubuntu/workspace/results/ab_car_validation_germany/docker/2023_07_26__20_32/b2b_det.tsv', sep='\t')
det['label'].value_counts()

In [11]:
import pandas as pd
import cv2
import os

root = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes'
det_path = f'{root}/co_detr.tsv'
img_path = '/home/ubuntu/workspace/datasets/ab_car_validation_germany/images/ab_car_munich_urban_day_009_001710.png'

det = pd.read_csv(det_path, sep='\t')
img = cv2.imread(img_path)
img_name = os.path.basename(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

for i, detection in det[det['name'] == img_name].iterrows():
    x_cent = int(detection['x_center'])
    y_cent = int(detection['y_center'])
    width = int(detection['width'])
    height = int(detection['height'])
    label = str(detection['label'])
    score = float(detection['score'])

    x1 = int(x_cent - width / 2)
    y1 = int(y_cent - height / 2)
    x2 = int(x_cent + width / 2)
    y2 = int(y_cent + height / 2)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    text = f"{label}: {score:.2f}"
    cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

output_path = f'{root}/output_image.png'
cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
print(output_path)


/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/output_image.png


In [12]:
import pickle

path = '/home/ubuntu/workspace/results/ab_car_validation_germany/faster_rcnn_cityscapes/backlog/Co-DETR/raw_detections/pickles/ab_car_munich_urban_day_009_001710.pickle'
with open(path, 'rb') as file:
    detections = pickle.load(file)
detections

[ObjectPrediction<
     bbox: BoundingBox: <(2246.798583984375, 916.0128173828125, 2337.927734375, 1088.1697998046875), w: 91.129150390625, h: 172.156982421875>,
     mask: None,
     score: PredictionScore: <value: 0.999940037727356>,
     category: Category: <id: 0, name: person>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1416.1263427734375, 980.6331787109375, 1456.6005859375, 1117.842529296875), w: 40.4742431640625, h: 137.2093505859375>,
     mask: None,
     score: PredictionScore: <value: 0.9858490228652954>,
     category: Category: <id: 0, name: person>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1296.5224609375, 1004.4595336914062, 1328.588623046875, 1052.45361328125), w: 32.066162109375, h: 47.99407958984375>,
     mask: None,
     score: PredictionScore: <value: 0.9828618764877319>,
     category: Category: <id: 0, name: person>>,
 ObjectPrediction<
     bbox: BoundingBox: <(1625.82958984375, 975.6094360351562, 1657.0850830078125, 1065.8294677734375), w: 31.2554931640